In [1]:
import mxnet as mx
from mxnet import gluon, nd, autograd
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms, CIFAR10
import time, os, shutil

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 16
trainset = CIFAR10(root = './data', train = True).transform_first(transform)
trainloader = gluon.data.DataLoader(trainset, batch_size = 16, shuffle = True, num_workers = batch_size)
testset = CIFAR10(root = './data', train = False).transform_first(transform)
testloader = gluon.data.DataLoader(testset, batch_size = 16, shuffle = True, num_workers = batch_size)

In [3]:
for features, labels in trainloader:
    print('features.min():', features.min().asscalar())
    print('features.max():', features.max().asscalar())
    
    print('labels:', labels)
    print('labels.min():', labels.min().asscalar())
    print('labels.max():', labels.max().asscalar())
    break

features.min(): -1.0
features.max(): 1.0
labels: 
[3 3 8 2 0 0 2 0 4 8 1 0 7 8 1 0]
<NDArray 16 @cpu_shared(0)>
labels.min(): 0
labels.max(): 8


In [4]:
class VGG(nn.Block):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = self._make_layers()
        self.classifier = nn.Dense(10)
    def forward(self, x):
        out = self.features(x)
        out = out.reshape((out.shape[0], -1))
        out = self.classifier(out)
        return out
    def _make_layers(self):
        layers = []
        in_channels = 3
        vgg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 
               512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
        for x in vgg:
            if x == 'M':
                layers += [nn.MaxPool2D(pool_size = 2, strides = 2)]
            else:
                layers += [nn.Conv2D(in_channels, kernel_size = 3, padding=1),
                           nn.BatchNorm(),
                           nn.Activation('relu')]
                in_channels = x
        layers += [nn.AvgPool2D(pool_size = 1, strides = 1)]
        model = nn.Sequential()
        model.add(*layers)
        return model
ctx = mx.cpu()
model = VGG()
model.initialize(ctx = ctx)
#print(model)

In [5]:
objective = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.001})

In [ ]:
losses = []
start = time.time()
for epoch in range(10):
    batch_start = time.time()
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        with autograd.record():
            outputs = model(inputs.as_in_context(ctx))
            loss = objective(outputs, labels.as_in_context(ctx)).mean()
        loss.backward()
        trainer.step(16)
        running_loss += loss
        losses.append(loss.asscalar())
    
    print(f'Epoch: {(epoch)} | Loss: {running_loss.asscalar() / (i + 1):.5f} | Images/Sec: {(batch_size * (i + 1)) / (time.time() - batch_start):.5f}')
        
        
    running_loss = 0.0
print('\nFinished Training')
print(f'Total time taken: {(time.time() - start)/60.0:.5f} minutes')